In [19]:
import importlib

# # # 1. 변경된 모듈을 평소처럼 import 합니다. (예: 'my_module.py'를 import)
import db_handler 

# # # 2. 모듈의 내용이 변경되었다면, 아래 코드를 실행하여 다시 로드합니다.
importlib.reload(db_handler)

from db_handler import DataBaseHandler

In [ ]:
# 있는지 체크하여 등록한다.

In [ ]:
zf_db = DataBaseHandler()

with zf_db as curr:
    curr.execute("""
select distinct *
from (
        select alt.batch_id,
            alt.batch_seq,
            alt.annc_url,
            alt.batch_status_cd,
            alt.batch_start_dttm,
            alt.batch_end_dttm,
            alt.annc_type,
            alt.annc_region,
            alt.annc_pblsh_dt,
            alt.annc_deadline_dt,
            alt.annc_status,
            alt.lh_pan_id,
            alt.lh_ais_tp_cd,
            alt.lh_upp_ais_tp_cd,
            alt.lh_ccr_cnnt_sys_ds_cd,
            alt.lh_ls_sst
        from annc_lh_temp alt
        where alt.batch_id = 'e4413166-2496-4f85-ac13-bac176e40ef0'
            and not exists(
                select *
                from annc_all aa
                where aa.annc_url = alt.annc_url
            )
        union all
        select alt.batch_id,
            alt.batch_seq,
            alt.annc_url,
            alt.batch_status_cd,
            alt.batch_start_dttm,
            alt.batch_end_dttm,
            alt.annc_type,
            alt.annc_region,
            alt.annc_pblsh_dt,
            alt.annc_deadline_dt,
            alt.annc_status,
            alt.lh_pan_id,
            alt.lh_ais_tp_cd,
            alt.lh_upp_ais_tp_cd,
            alt.lh_ccr_cnnt_sys_ds_cd,
            alt.lh_ls_sst
        from annc_lh_temp alt
            join annc_all aa on alt.annc_url = aa.annc_url
        where alt.annc_pblsh_dt != aa.annc_pblsh_dt
            or alt.annc_pblsh_dt != aa.annc_pblsh_dt
    ) a
""")
    all_data = curr.fetchall()

In [25]:
all_data

('e4413166-2496-4f85-ac13-bac176e40ef0',
 2298,
 'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=BN-0006739&ccrCnntSysDsCd=01&uppAisTpCd=01&aisTpCd=01',
 'READY',
 None,
 None,
 '토지',
 '인천광역시',
 '2025.02.18',
 '2025.03.19',
 '접수마감',
 'BN-0006739',
 '01',
 '01',
 '01',
 '')

In [ ]:
# all_data[0]['batch_id']

TypeError: tuple indices must be integers or slices, not str

In [4]:
batch_size = 7

1000 // batch_size, 1000 % batch_size

(142, 6)

In [ ]:
total_cnt = 3078

batch_size = 100

list_sample = [(i+1, f"{i+1}번째 ") for i in range(total_cnt)]



    # print(start_idx, end_idx)
    # print(list_sample[start_idx:end_idx])

0 100
100 200
200 300
300 400
400 500
500 600
600 700
700 800
800 900
900 1000
1000 1100
1100 1200
1200 1300
1300 1400
1400 1500
1500 1600
1600 1700
1700 1800
1800 1900
1900 2000
2000 2100
2100 2200
2200 2300
2300 2400
2400 2500
2500 2600
2600 2700
2700 2800
2800 2900
2900 3000
3000 3078


In [ ]:
def bulk_merge_annc_all(data_list, batch_size=100):

    # MERGE 쿼리 템플릿
    # {values_sql_placeholder} 위치에 동적으로 생성된 VALUES 구문이 들어갑니다.
    merge_query_template = """
        MERGE INTO ANNC_ALL AS Target
        USING (
            VALUES {values_sql_placeholder}
        ) AS Source (ANNC_URL, CORP_CD, ANNC_TYPE, ANNC_REGION, ANNC_PBLSH_DT, ANNC_DEADLINE_DT, ANNC_STATUS, SERVICE_STATUS)
        ON (Target.ANNC_URL = Source.ANNC_URL) -- ANNC_URL을 기준으로 일치 여부 확인
        
        -- 일치하는 행이 있으면 UPDATE
        WHEN MATCHED THEN
            UPDATE SET 
                ANNC_TYPE = Source.ANNC_TYPE,
                ANNC_STATUS = Source.ANNC_STATUS,
                IS_USE = Source.IS_USE,
                ANNC_PBLSH_DT = Source.ANNC_PBLSH_DT,
                ANNC_DEADLINE_DT = Source.ANNC_DEADLINE_DT

        -- 일치하는 행이 없으면 INSERT
        WHEN NOT MATCHED THEN
            INSERT (ANNC_URL, CORP_CD, ANNC_TYPE, ANNC_REGION, ANNC_PBLSH_DT, ANNC_DEADLINE_DT, ANNC_STATUS, SERVICE_STATUS)
            VALUES (Source.ANNC_URL, Source.CORP_CD, Source.ANNC_TYPE, Source.ANNC_REGION, Source.ANNC_RBLSH_DT, Source.ANNC_DEADLINE_DT, Source.ANNC_STATUS, Source.SERVICE_STATUS);
    """

    total_cnt = len(data_list)

    for i in range((total_cnt // batch_size) + 1):
        batch_list = data_list[i:i + batch_size]

        
        

    # for _ in range()

In [ ]:
def bulk_merge_annc_all(data_list, chunk_size):
    """데이터 리스트를 청크 단위로 분할하여 MERGE 쿼리를 실행합니다."""

    # MERGE 쿼리 템플릿
    # {values_sql_placeholder} 위치에 동적으로 생성된 VALUES 구문이 들어갑니다.
    merge_query_template = """
        MERGE INTO ANNC_ALL AS Target
        USING (
            VALUES {values_sql_placeholder}
        ) AS Source (ANNC_URL, CORP_CD, ANNC_TYPE, ANNC_REGION, ANNC_RBLSH_DT, ANNC_DEADLINE_DT, ANNC_STATUS, IS_USE)
        ON (Target.ANNC_URL = Source.ANNC_URL) -- ANNC_URL을 기준으로 일치 여부 확인
        
        -- 일치하는 행이 있으면 UPDATE
        WHEN MATCHED THEN
            UPDATE SET 
                ANNC_TYPE = Source.ANNC_TYPE,
                ANNC_STATUS = Source.ANNC_STATUS,
                IS_USE = Source.IS_USE,
                ANNC_DEADLINE_DT = Source.ANNC_DEADLINE_DT

        -- 일치하는 행이 없으면 INSERT
        WHEN NOT MATCHED THEN
            INSERT (ANNC_URL, CORP_CD, ANNC_TYPE, ANNC_REGION, ANNC_RBLSH_DT, ANNC_DEADLINE_DT, ANNC_STATUS, IS_USE)
            VALUES (Source.ANNC_URL, Source.CORP_CD, Source.ANNC_TYPE, Source.ANNC_REGION, Source.ANNC_RBLSH_DT, Source.ANNC_DEADLINE_DT, Source.ANNC_STATUS, Source.IS_USE);
    """

    with psycopg.connect(DB_CONFIG) as conn:
        with conn.cursor() as cur:

            # 데이터를 청크 단위로 묶어 반복 처리
            for i in range(0, len(data_list), chunk_size):
                chunk = data_list[i : i + chunk_size]

                # VALUES 절에 들어갈 데이터 (튜플 리스트)
                values_to_insert = [
                    (
                        item['ANNC_URL'],
                        item['CORP_CD'],
                        item['ANNC_TYPE'],
                        item['ANNC_REGION'],
                        item['ANNC_RBLSH_DT'],
                        item['ANNC_DEADLINE_DT'],
                        item['ANNC_STATUS'],
                        item['IS_USE']
                    ) for item in chunk
                ]

                # Psycopg를 사용하여 VALUES 구문을 안전하게 생성
                # 예: ('url1', 'LH', ...), ('url2', 'LH', ...)
                value_placeholders = "(" + ", ".join(["%s"] * len(values_to_insert[0])) + ")"
                
                # 모든 VALUES 튜플을 합친 단일 SQL 문자열 생성
                values_sql = ", ".join([value_placeholders] * len(values_to_insert))
                
                # 최종 쿼리에 VALUES 구문 삽입
                final_query = merge_query_template.replace("{values_sql_placeholder}", values_sql)

                # 모든 청크 데이터의 값을 단일 리스트로 펼치기 (flat list)
                flat_values = [val for row in values_to_insert for val in row]

                # 쿼리 실행
                cur.execute(final_query, flat_values)
                print(f"✅ {i}부터 {i + len(chunk) - 1}까지 {len(chunk)}건 MERGE 완료.")

        conn.commit()

In [ ]:
for data in all_data:

    corp_cd = 'LH'

    batch_id = data[0]
    batch_seq = data[1]
    annc_url = data[2]
    batch_status_cd = data[3]
    batch_start_dttm = data[4]
    batch_end_dttm = data[5]
    annc_type = data[6]
    annc_region = data[7]
    annc_pblsh_dt = data[8]
    annc_deadline_dt = data[9]
    annc_status = data[10]
    lh_pan_id = data[11] 
    lh_ais_tp_cd = data[12]
    lh_upp_ais_tp_cd = data[13]
    lh_ccr_cnnt_sys_ds_cd = data[14]
    lh_ls_sst = data[15]


    ANNC_ALL = f"""
    INSERT INTO ANNC_ALL (
        ANNC_URL, CORP_CD, ANNC_TYPE, ANNC_REGION, ANNC_PBLSH_DT, ANNC_DEADLINE_DT, ANNC_STATUS, SERVICE_STATUS
    ) VALUES (
        '{annc_url}', '{corp_cd}', '{annc_type}', '{annc_region}', '{annc_pblsh_dt}', '{annc_deadline_dt}', '{annc_status}', 'Y'
    );
    """



    ...

[('e4413166-2496-4f85-ac13-bac176e40ef0',
  2298,
  'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=BN-0006739&ccrCnntSysDsCd=01&uppAisTpCd=01&aisTpCd=01',
  'READY',
  None,
  None,
  '토지',
  '인천광역시',
  '2025.02.18',
  '2025.03.19',
  '접수마감',
  'BN-0006739',
  '01',
  '01',
  '01',
  ''),
 ('e4413166-2496-4f85-ac13-bac176e40ef0',
  2031,
  'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=BN-0006796&ccrCnntSysDsCd=01&uppAisTpCd=01&aisTpCd=01',
  'READY',
  None,
  None,
  '토지',
  '인천광역시',
  '2025.03.24',
  '2025.03.24',
  '접수마감',
  'BN-0006796',
  '01',
  '01',
  '01',
  ''),
 ('e4413166-2496-4f85-ac13-bac176e40ef0',
  354,
  'https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?mi=1026&panId=2015122300018841&ccrCnntSysDsCd=03&uppAisTpCd=06&aisTpCd=07',
  'READY',
  None,
  None,
  '국민임대',
  '경기도',
  '2025.09.26',
  '2025.10.31',
  '접수마감',
  '2015122300018841',
  '07',
  '06',
  '03',
  ''),
 ('e4413166-24